# Описание проекта

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [2]:
geo_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(geo_1.head(10))
print(geo_2.head(10))
print(geo_3.head(10))


      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
5  HHckp  -3.327590  -2.205276  3.003647   84.038886
6  h5Ujo -11.142655 -10.133399  4.002382  110.992147
7  muH9x   4.23

In [4]:
print(geo_1.info())
print(geo_2.info())
print(geo_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [5]:
print(geo_1.isnull().sum())
print(geo_2.isnull().sum())
print(geo_3.isnull().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [6]:
print(geo_1.duplicated().sum())
print(geo_2.duplicated().sum())
print(geo_3.duplicated().sum())

0
0
0


#### Я загрузил 3 датасета разных локаций, просмотрел первые 10 строк и информацию датасетов с помощью Info.

#### проверил данные на пропуски и дубликаты, таковых не оказалось. На мой взгляд данные полностью готовы к работе.

# 2. Обучение и проверка модели

### 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

In [8]:
target_1 = geo_1['product']
features_1 = geo_1.drop(['product', 'id'], axis = 1)

target_2 = geo_2['product']
features_2 = geo_2.drop(['product', 'id'], axis = 1)

target_3 = geo_3['product']
features_3 = geo_3.drop(['product', 'id'], axis = 1)

In [9]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size = 0.25, random_state = 12345)

In [10]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size = 0.25, random_state = 12345)

In [11]:
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(
    features_3, target_3, test_size = 0.25, random_state = 12345)

#### Я загрузил train_test_split и с его помощью разбил выборку на валидационную и тренировочную в необходимых пропорциях.

### 2.2. Обучите модель и сделайте предсказания на валидационной выборке.

In [13]:
model_1 = LinearRegression().fit(features_train_1, target_train_1)
model_2 = LinearRegression().fit(features_train_2, target_train_2)
model_3 = LinearRegression().fit(features_train_3, target_train_3)

In [14]:
predicted_valid_1 = model_1.predict(features_valid_1)
print('R2 Score_1', r2_score(target_valid_1, predicted_valid_1))
mse_1 = mean_squared_error(target_valid_1, predicted_valid_1)
print('MSE_1', mse_1)
print('RMSE_1', mse_1**0.5)
print('средний запас 1', target_valid_1.mean())

R2 Score_1 0.27994321524487786
MSE_1 1412.2129364399243
RMSE_1 37.5794217150813
средний запас 1 92.07859674082927


In [15]:
predicted_valid_2 = model_2.predict(features_valid_2)
print('R2 Score_2', r2_score(target_valid_2, predicted_valid_2))
mse_2 = mean_squared_error(target_valid_2, predicted_valid_2)
print('MSE_2', mse_2)
print('RMSE_2', mse_2**0.5)
print('средний запас 2', target_valid_2.mean())

R2 Score_2 0.9996233978805127
MSE_2 0.7976263360391139
RMSE_2 0.893099286775616
средний запас 2 68.72313602435997


In [16]:
predicted_valid_3 = model_3.predict(features_valid_3)
print('R2 Score_3', r2_score(target_valid_3, predicted_valid_3))
mse_3 = mean_squared_error(target_valid_3, predicted_valid_3)
print('MSE_3', mse_3)
print('RMSE_3', mse_3**0.5)
print('средний запас 3', target_valid_3.mean())

R2 Score_3 0.20524758386040443
MSE_3 1602.3775813236196
RMSE_3 40.02970873393434
средний запас 3 94.88423280885438


#### По первой и третьей локации примерна одинаковая сиуация, RMSE около 35% от среднего показателя. 

#### На мой взгляд модели конечно не идеальны, но результат выше 50%, что вероятно говорит об адекватности моделей

#### С второй локацией и моделью по ней я получил необычайно точную модель с r2 практически 1, соответсввенно и ошибок в ней практически нет.

# 3. Подготовка к расчёту прибыли

### 3.1. Сохраните в коде все ключевые значения для расчётов.

In [18]:
budjet = 10000000000
well_drill = 50000000
per_bar = 4500

#### предоставленные мне данные (бюджет, трата на бурение скважины, цена за баррель) я сохранил в переменных

### 3.2. Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.

In [19]:
print('Количество скважин, которые компания может пробурить при текущем буджете :', int(budjet / well_drill))
print('минимальные запасы нефти месторождения, окупающие его бурение :', well_drill/per_bar)

Количество скважин, которые компания может пробурить при текущем буджете : 200
минимальные запасы нефти месторождения, окупающие его бурение : 11111.111111111111


#### При текущем бюджете и тратах, компания может позволить себе 200 скважин, запас которых должен быть выже 11111 баррелей 

### 3.3. Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.

In [28]:
def revenue(target, predict, count):
    predicted = predict.sort_values(ascending = False)
    selected = target[predicted.index][:count]
    return per_bar * 1000 * selected.sum() - budjet



#### done

# 4. Расчёт прибыли и рисков 

In [21]:
state = RandomState(12345)


### 4.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [22]:
target_valid_1 = pd.Series(target_valid_1)
predicted_valid_1 = pd.Series(predicted_valid_1)
values = []
for i in range(1000):
    target_subsample = target_valid_1.sample(n=500, replace=True, random_state=state)
    pred_subsample = predicted_valid_1[target_subsample.index]
    values.append(revenue(target_subsample, pred_subsample, 200))

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


#### используя функцию полученную выше, я применил бутстреп к предсказаниям полученными моделью по первой локации. в параметрах функции я указал количество меторождений исходя из бюджета - 200

### 4.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [23]:
values = pd.Series(values)
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
print('95% доверительный интервал локации 1 :', confidence_interval)
print('Средняя выручка локации 1 :', values.mean())

95% доверительный интервал локации 1 : (72549161014.92882, 72909357896.74953)
Средняя выручка локации 1 : 72729259455.83917


#### Я получил доверительный интервал 72.5 - 72.9 миллиарда. Мне кажется что риска убытков нет.

### Повторю прерыдущие пункты на остальных локациях:

In [24]:

target_valid_2 = pd.Series(target_valid_2)
predicted_valid_2 = pd.Series(predicted_valid_2)
values_2 = []
for i in range(1000):
    target_subsample2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    pred_subsample2 = predicted_valid_2[target_subsample2.index]
    values_2.append(revenue(target_subsample2, pred_subsample2, 200))

In [25]:
values_2 = pd.Series(values_2)
confidence_interval_2 = st.t.interval(0.95, len(values_2)-1, values_2.mean(), values_2.sem())
print('95% доверительный интервал локации 2 :', confidence_interval_2)
print('Средняя выручка локации 2 :', values_2.mean())

95% доверительный интервал локации 2 : (51701061547.19408, 52068642354.0863)
Средняя выручка локации 2 : 51884851950.64019


In [ ]:

target_valid_3 = pd.Series(target_valid_3)
predicted_valid_3 = pd.Series(predicted_valid_3)
values_3 = []
for i in range(1000):
    target_subsample3 = target_valid_3.sample(n=500, replace=True, random_state=state)
    pred_subsample3 = predicted_valid_3[target_subsample3.index]
    values_3.append(revenue(target_subsample3, pred_subsample3, 200))

In [27]:
values_3 = pd.Series(values_3)
confidence_interval_3 = st.t.interval(0.95, len(values_3)-1, values_3.mean(), values_3.sem())
print('95% доверительный интервал локации 3 :', confidence_interval_3)
print('Средняя выручка локации 3 :', values_3.mean())

95% доверительный интервал локации 3 : (75011491431.25368, 75380153097.89415)
Средняя выручка локации 3 : 75195822264.57391


### Вывод
#### Из полученных данных я могу судить что все регионы прибыльны и риска убытков нет. Но выбрать для освоения стоит 3ю локацию, тк ожидаемая прибыль с нее больше остальных, и доверительный интервал меньше, более предсказуемый результат.